In [12]:
import requests
import gzip
import pandas as pd
import shutil
import datetime

In [13]:
ftpstream = requests.get("https://smn.conagua.gob.mx/webservices/?method=3", headers={'User-Agent': 'Mozilla/5.0'})

In [14]:
type(ftpstream.content)

bytes

In [15]:
with open('./file.gz', 'wb') as file:
    file.write(ftpstream.content)

In [16]:
with gzip.open('./file.gz', 'rb') as f_in:
    with open('response.json', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


In [17]:
df = pd.read_json('./response.json')

In [18]:
df

,desciel,dh,dirvienc,dirvieng,dpt,dsem,hloc,hr,ides,idmun,lat,lon,nes,nhor,nmun,prec,probprec,raf,temp,velvien
0,Cielo nublado,6,Noreste,51.23,12.56,Miércoles,20230412T07,88.27,20,54,17.3884,-97.2290,Oaxaca,-1,Magdalena Zahuatlán,0.00,80.0,6.33,13.39,5.35
1,Cielo nublado,6,Noreste,48.64,13.61,Miércoles,20230412T08,77.03,20,54,17.3884,-97.2290,Oaxaca,0,Magdalena Zahuatlán,0.00,0.0,4.92,15.37,4.90
2,Cielo nublado,6,Noreste,66.28,15.13,Miércoles,20230412T09,64.50,20,54,17.3884,-97.2290,Oaxaca,1,Magdalena Zahuatlán,0.00,0.0,3.54,18.13,2.98
3,Medio nublado,6,Sureste,119.53,16.52,Miércoles,20230412T10,52.33,20,54,17.3884,-97.2290,Oaxaca,2,Magdalena Zahuatlán,0.07,0.0,5.57,21.03,2.43
4,Medio nublado,6,Sur,194.14,17.47,Miércoles,20230412T11,42.87,20,54,17.3884,-97.2290,Oaxaca,3,Magdalena Zahuatlán,0.12,20.0,8.99,23.44,3.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285703,Despejado,6,Este,106.41,15.50,Domingo,20230416T22,54.25,19,46,25.7553,-100.2896,Nuevo León,110,San Nicolás de los Garza,0.00,0.0,27.70,19.71,16.64
285704,Despejado,6,Sureste,117.68,14.98,Domingo,20230416T23,59.22,19,46,25.7553,-100.2896,Nuevo León,111,San Nicolás de los Garza,0.00,0.0,26.18,18.57,15.55
285705,Despejado,6,Sureste,122.94,14.59,Lunes,20230417T00,63.14,19,46,25.7553,-100.2896,Nuevo León,112,San Nicolás de los Garza,0.00,0.0,25.69,17.73,14.38
285706,Despejado,6,Sureste,121.57,14.25,Lunes,20230417T01,66.89,19,46,25.7553,-100.2896,Nuevo León,113,San Nicolás de los Garza,0.00,0.0,22.17,16.98,12.19


In [20]:
df[['fecha', 'hora']] = df['hloc'].str.split('T', expand=True)
df['fecha'] = pd.to_datetime(df['fecha'])

In [21]:
df['hora'] = df['hora'].astype('int')

In [22]:
df['fecha'].max()

Timestamp('2023-04-17 00:00:00')

In [24]:
df_2 = df[(df['fecha'] == datetime.date.today().isoformat()) & 
   (abs(df['hora'] - datetime.datetime.today().hour) < 3) & 
   (df['hora'] <= datetime.datetime.today().hour)].groupby(['ides', 'idmun']).mean()[['temp', 'prec']]


/tmp/ipykernel_1024549/1255087492.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_2 = df[(df['fecha'] == datetime.date.today().isoformat()) &


In [25]:
df_2 = df_2.reset_index()

In [26]:
df_2

,ides,idmun,temp,prec
